In [17]:
#URL
# Marvel
# https://brickset.com/sets/theme-Marvel-Super-Heroes

# Harry Potter 
# https://brickset.com/sets/theme-Harry-Potter

# Star Wars
# https://brickset.com/sets/theme-Star-Wars


In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

driver = webdriver.Chrome() 

# URL base para la categoría
base_url = "https://brickset.com/sets/theme-Harry-Potter/page-{}"
sets_data = []


In [12]:
# Función para extraer el código del set
def extract_set_code(title):
    match = re.match(r'(\d+):', title)
    return match.group(1) if match else "Código no disponible"

In [13]:
# Función para realizar scraping de cada página con Selenium
def scrape_page(page_num):
    url = base_url.format(page_num)
    driver.get(url)
    
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "set")))

        # BeautifulSoup para extraer datos de la página
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        sets = soup.find_all('article', class_='set')
        
        # Si no encuentra sets, detener la búsqueda
        if not sets:
            print(f"No se encontraron sets en la página {page_num}. Deteniendo.")
            return False
        
        print(f"Página {page_num} contiene {len(sets)} sets.")  # Información de depuración
        
        for set_item in sets:
            # Obtener el nombre del set
            title_tag = set_item.find('div', class_='meta').find('h1').find('a')
            title = title_tag.text.strip() if title_tag else "Título no disponible"
            
            # Extraer el código del set del título
            set_code = extract_set_code(title)
            
            # Año de lanzamiento después de "Launch/exit"
            launch_tag = set_item.find('dt', string="Launch/exit")
            year = launch_tag.find_next_sibling('dd').text.strip() if launch_tag else "Año no disponible"

            # PVP
            pvp_tag = set_item.find('dt', string="RRP")
            pvp = pvp_tag.find_next_sibling('dd').text.strip() if pvp_tag else "PVP no disponible"
            
            # Precio como nuevo
            new_value_tag = set_item.find('dt', string="Value new")
            price_new = new_value_tag.find_next_sibling('dd').text.strip() if new_value_tag else "Precio nuevo no disponible"
            
            # Precio como usado
            used_value_tag = set_item.find('dt', string="Value used")
            price_used = used_value_tag.find_next_sibling('dd').text.strip() if used_value_tag else "Precio usado no disponible"
            
            # Valoración de usuarios
            rating_tag = set_item.find('div', class_='rating')
            rating = rating_tag.text.strip() if rating_tag else "Valoración no disponible"
            
            # Agrego los datos a la lista
            sets_data.append({
                'Código del Set': set_code,
                'Nombre del Set': title,
                'Año de Lanzamiento': year,
                'PVP':pvp,
                'Precio como Nuevo': price_new,
                'Precio como Usado': price_used,
                'Valoración de Usuarios': rating,
            })
        
    except Exception as e:
        print(f"Error en la página {page_num}: {e}")
        return False
    
    return True

# Iterar sobre todas las páginas hasta que no haya más sets
page_num = 1
while True:
    print(f"Scraping página {page_num}...")
    if not scrape_page(page_num):
        break
    page_num += 1
    time.sleep(5)  # Pausa de 5 segundos para evitar sobrecargar el servidor

# Cerrar el navegador
driver.quit()


Scraping página 1...
Página 1 contiene 25 sets.
Scraping página 2...
Página 2 contiene 25 sets.
Scraping página 3...
Página 3 contiene 25 sets.
Scraping página 4...
Página 4 contiene 25 sets.
Scraping página 5...
Página 5 contiene 25 sets.
Scraping página 6...
Página 6 contiene 25 sets.
Scraping página 7...
Página 7 contiene 24 sets.
Scraping página 8...
Error en la página 8: Message: 



In [14]:
# Creo DataFrame con los datos recopilados
df_brickset = pd.DataFrame(sets_data)

# Guardo el DataFrame en un archivo CSV
df_brickset.to_csv('lego_harry_potter_sets.csv', index=False)

In [15]:
print(f"Total de sets recopilados: {len(df_brickset)}")

Total de sets recopilados: 174


In [16]:
df_brickset

,Código del Set,Nombre del Set,Año de Lanzamiento,PVP,Precio como Nuevo,Precio como Usado,Valoración de Usuarios
0,4695,4695: Mini Harry Potter Knight Bus,1 Apr 2004 - 31 Dec 2004 (9m),PVP no disponible,~€21.74,~€8.06,✭✭✭✭✩ 3.6 64 ratings 11 reviews
1,4701,4701: Sorting Hat,1 Sep 2001 - 31 Dec 2002 (1y 3m 29d),$7.00,~€79.69,~€12.33,✭✭✭✭✩ 3.4 154 ratings 19 reviews
2,4702,4702: The Final Challenge,1 Sep 2001 - 31 Dec 2002 (1y 3m 29d),$10.00,~€40.89,~€16.08,✭✭✭✭✩ 3.6 154 ratings 14 reviews
3,4704,4704: The Chamber of the Winged Keys,1 Sep 2001 - 31 Dec 2002 (1y 3m 29d),$20.00,~€83.37,~€31.93,✭✭✭✭✩ 3.7 95 ratings 9 reviews
4,4705,4705: Snape's Class,1 Sep 2001 - 31 Dec 2002 (1y 3m 29d),$20.00,~€61.57,~€28.65,✭✭✭✭✩ 3.8 121 ratings 13 reviews
...,...,...,...,...,...,...,...
169,Código no disponible,Knight Bus,Año no disponible,PVP no disponible,Precio nuevo no disponible,Precio usado no disponible,
170,Código no disponible,Voldemort,Año no disponible,$0.00,Precio nuevo no disponible,Precio usado no disponible,
171,Código no disponible,Golden Snitch,Año no disponible,PVP no disponible,~€9.49,~€11.31,
172,Código no disponible,Quidditch Tower (TRU Event Exclusive),Año no disponible,PVP no disponible,Precio nuevo no disponible,~€21.98,1 review
